This will train the network I guess
And then I will likely need another file to test the network

# Import

In [1]:
import os
import sys
import time
import glob
import torch
import utils
import logging
import argparse
from pcanet import PCANet
from dataset_mnist import load_train_mnist
from sklearn.svm import LinearSVC, SVC
from tensorboardX import SummaryWriter
from torchvision.utils import make_grid
from utils import MyLogger

os.environ["CUDA_VISIBLE_DEVICES"] = "0"

parser = argparse.ArgumentParser("PCANet")
# this one fixes the issue in ipython apparently
# https://stackoverflow.com/questions/42249982/systemexit-2-error-when-calling-parse-args-within-ipython
parser.add_argument('-f')
parser.add_argument('--gpu', type=int, default=0, help='gpu device id')
parser.add_argument('--dataset_name', type=str, default='mnist', help='mnist or cifar10')
parser.add_argument('--dataset_path', type=str, default='/dataset/', help='location of the data corpus')
parser.add_argument('--batch_size', type=int, default=128, help='batch size')
# this train portion can be modified to make it faster i guess, default is 1.0
parser.add_argument('--train_portion', type=float, default=0.01, help='portion of training data')
parser.add_argument('--stages', type=int, default=2, help='the number of stages')
parser.add_argument('--filter_shape', type=list, default=[7, 7], help='patch size')
parser.add_argument('--stages_channels', type=list, default=[8, 8], help='channels in different stages')
parser.add_argument('--block_size', type=int, default=7, help='the size of blocks')
parser.add_argument('--block_overlap', type=float, default=0.5, help='the rate of overlap between blocks')
parser.add_argument('--save', type=str, default='EXP', help='experiment name')
parser.add_argument('--log_freq', type=int, default=40, help='record log frequency')
args = parser.parse_args()

args.save = 'search-{}-{}'.format(args.save, time.strftime("%Y%m%d-%H%M%S"))
# seems like this is the one that creates a duplicated python file for some reason, most likely to keep a copy of whatever code is executed for easier debugging maybe?
utils.create_exp_dir(args.save, scripts_to_save=glob.glob('*.py'))

logger = MyLogger("my_log.log")

CLASSES = 10

Experiment dir : search-EXP-20230815-103200


# Training section


Output images are of batch, channel, width, height

In [ ]:
from torch.utils.data import DataLoader, Dataset

# the custom dataset for the headlight
class HeadlightCustomDataset(Dataset):
    '''
    this will be for the custom headlight dataset generated.
    It must return the image, as well as the class label

    for the class label, it must find the corresponding file name in
    the original source file, and them match it based on the subfolder
    i guess
    '''
    def __init__(self, annotations_dir, img_dir, annotation_label_dir, transform=None, target_transform=None, split=None):
        import os
        self.annotations_dir = annotations_dir
        self.img_dir = img_dir
        self.transform = transform
        self.target_transform = target_transform
        self.annotation_label_dir = annotation_label_dir

        # count number of annotations there are
        if split is None:
            # this means that there is no splitting of whatever
            self.ann_files = []
            for _, _, file in os.walk(annotations_dir):
                self.ann_files.extend(file)
        else:
            # this one should just read the split file i guess
            self.ann_files = split

        # get the annotation names
        self.ann_labels = []
        with open(self.annotation_label_dir, 'r') as file:
            for line in file:
                self.ann_labels.append(line.strip())

    def __len__(self):
        return len(self.ann_files)

    def __getitem__(self, idx):
        # returns the annotations as well as the image needed
        import pandas as pd
        from torchvision.io import read_image
        import json

        img_name = self.ann_files[idx].replace('.json', '') + '.jpg'

        img_path = os.path.join(self.img_dir, img_name)
        image = read_image(img_path)

        label_path = os.path.join(self.annotations_dir, self.ann_files[idx])
        with open(label_path, 'r') as file:
            label = json.load(file)

        # converting the labels from h, w, x, y to x1, y1, x2, y2
        # planning to output in format of x1, y1, x2, y2, label
        converted_labels = []
        for i in label['annotations']:
            # convert the annotation NAME to annotation NUMBER
            for index, keyword in enumerate(self.ann_labels):
                if keyword == i['name']:
                    temp_label = index + 1
                    break

            # convert the coordinates
            temp = [i['bounding_box']['x'],
                    i['bounding_box']['y'],
                    i['bounding_box']['x'] + i['bounding_box']['w'],
                    i['bounding_box']['y'] + i['bounding_box']['h'],
                    temp_label]
            converted_labels.append(temp)

        if self.transform:
            image = self.transform(image)
        if self.target_transform:
            converted_labels = self.target_transform(converted_labels)
        return image, converted_labels

In [2]:

if not torch.cuda.is_available():
    logger.log('no gpu device available')
    sys.exit(1)

logger.log('gpu device = %d' % args.gpu)
temp_log_string = "args = %s", args
logger.log(temp_log_string)

pcanet = PCANet(args.stages, args.filter_shape, args.stages_channels, args.block_size, args.block_overlap)
train_queue, valid_queue = load_train_mnist(args)        # load dataset
logger.log("load training dataset completely")
total_train_labels = torch.tensor([]).long()

writer = SummaryWriter(args.save)       # tensorboardX

counter = 0
for global_step, (train_images, train_labels) in enumerate(train_queue):
    print('test')
    counter += 1
    if counter == 5:
        break

gpu device = 0
('args = %s', Namespace(f='C:\\Users\\PC\\AppData\\Roaming\\jupyter\\runtime\\kernel-8ed295ff-62d8-41fe-90fd-7c83154a6fd9.json', gpu=0, dataset_name='mnist', dataset_path='/dataset/', batch_size=128, train_portion=0.01, stages=2, filter_shape=[7, 7], stages_channels=[8, 8], block_size=7, block_overlap=0.5, save='search-EXP-20230815-103200', log_freq=40))
load training dataset completely
test
test
test
test
test
